### Code which statisfies only the major requirements in assignmnet

In [3]:
import pandas as pd
from sklearn.metrics import mean_squared_error
import numpy as np
from bokeh.plotting import figure, output_file, save
from bokeh.models import ColumnDataSource
import unittest
import webbrowser
import os

# Correct file paths (Ensure these directories exist)
train_file_path = r'C:\Users\m.ranka\Desktop\IU\Python\Datasets1\train.csv'
ideal_file_path = r'C:\Users\m.ranka\Desktop\IU\Python\Datasets1\ideal.csv'
test_file_path = r'C:\Users\m.ranka\Desktop\IU\Python\Datasets1\test.csv'

# Load datasets
training_data = pd.read_csv(train_file_path)
ideal_functions = pd.read_csv(ideal_file_path)
test_data = pd.read_csv(test_file_path)

# Step 1: Select the four best ideal functions using Least-Squares Method
def select_best_ideal_functions(training_data, ideal_functions):
    best_functions = {}
    log_results = []

    for i in range(1, 5):
        y_train = training_data[f'y{i}'].values
        mse_list = []

        for j in range(1, 51):  # 50 ideal functions
            y_ideal = ideal_functions[f'y{j}'].values
            mse = mean_squared_error(y_train, y_ideal)
            mse_list.append((mse, f'y{j}'))

        # Find the best ideal function with the lowest MSE
        best_function = min(mse_list, key=lambda x: x[0])
        best_functions[f'y{i}'] = best_function[1]
        log_results.append({
            'training_function': f'y{i}',
            'best_ideal_function': best_function[1],
            'mse': best_function[0]
        })

    # Convert log results to DataFrame for easy output
    log_df = pd.DataFrame(log_results)
    return best_functions, log_df

# Step 2: Map the test data to the selected ideal functions with full log
def map_test_data_with_full_log(test_data, ideal_functions, best_functions, training_data):
    full_log = []
    mapped_data = []

    for index, row in test_data.iterrows():
        x_test, y_test = row['x'], row['y']
        assigned = False
        log_entry = {
            'x': x_test,
            'y_test': y_test,
            'y_ideal': None,
            'ideal_function': None,
            'deviation': None,
            'threshold': None,
            'status': 'Not Assigned - Exceeds Threshold'
        }
        
        for train_key, ideal_key in best_functions.items():
            y_ideal = ideal_functions[ideal_functions['x'] == x_test][ideal_key].values
            if y_ideal.size > 0:
                y_ideal_value = y_ideal[0]
                deviation = abs(y_test - y_ideal_value)

                y_train_value = training_data[training_data['x'] == x_test][train_key].values[0]
                max_deviation_train = abs(y_train_value - y_ideal_value)
                threshold = np.sqrt(2) * max_deviation_train

                log_entry.update({
                    'y_ideal': y_ideal_value,
                    'ideal_function': ideal_key,
                    'deviation': deviation,
                    'threshold': threshold
                })

                if deviation <= threshold:
                    log_entry['status'] = 'Assigned'
                    mapped_data.append({
                        'x': x_test,
                        'y_test': y_test,
                        'y_ideal': y_ideal_value,
                        'ideal_function': ideal_key,
                        'deviation': deviation,
                        'threshold': threshold
                    })
                    assigned = True
                    break

        full_log.append(log_entry)

    return pd.DataFrame(full_log), pd.DataFrame(mapped_data)

# Step 3: Visualize the results using Bokeh with Different Colors for Threshold Satisfied/Exceeded
def visualize_results_with_colors(training_data, ideal_functions, mapped_results, best_functions):
    html_path = r"C:\Users\m.ranka\Desktop\IU\Python\Datasets1\ideal_functions_and_mapping_visualization.html"  # Local path
    output_file(html_path)
    
    p = figure(title="Training Data, Ideal Functions, and Mapped Test Data with Color Coding",
               x_axis_label="X", y_axis_label="Y", width=800, height=600)

    # Plot training data
    for i in range(1, 5):
        p.scatter(training_data['x'], training_data[f'y{i}'], legend_label=f"Training y{i}", size=5, color='blue', alpha=0.6)

    # Plot ideal functions
    for train_key, ideal_key in best_functions.items():
        p.line(ideal_functions['x'], ideal_functions[ideal_key], legend_label=f"Ideal {ideal_key}", line_width=2, color='green')

    # Separate the test data into "Assigned" and "Not Assigned" (based on threshold)
    source_assigned = ColumnDataSource(mapped_results[mapped_results['status'] == 'Assigned'])
    source_not_assigned = ColumnDataSource(mapped_results[mapped_results['status'] != 'Assigned'])

    # Plot points that satisfy the threshold (Assigned) in green
    p.scatter('x', 'y_test', legend_label="Mapped Test Data (Within Threshold)", source=source_assigned, size=8, color='green', alpha=0.6)

    # Plot points that exceed the threshold (Not Assigned) in red
    p.scatter('x', 'y_test', legend_label="Test Data (Exceeds Threshold)", source=source_not_assigned, size=8, color='red', alpha=0.6)

    save(p)
    
    # Automatically open the HTML file in the browser
    webbrowser.open(html_path)

# Find best ideal functions
best_ideal_functions, log_results_df = select_best_ideal_functions(training_data, ideal_functions)

# Map test data and log all deviations
mapped_results_df, mapped_data_df = map_test_data_with_full_log(test_data, ideal_functions, best_ideal_functions, training_data)

# Step 4: Save results to Excel (Ensure the directory exists)
output_excel_path = r'C:\Users\m.ranka\Desktop\IU\Python\Datasets1\best_ideal_functions_with_full_log.xlsx'  # Local path
with pd.ExcelWriter(output_excel_path) as writer:
    log_results_df.to_excel(writer, sheet_name='Best Ideal Functions Log', index=False)
    mapped_results_df.to_excel(writer, sheet_name='Full Log with Deviations', index=False)
    mapped_data_df.to_excel(writer, sheet_name='Mapped Test Data', index=False)

# Automatically open the Excel file
os.startfile(output_excel_path)

# Call the visualization function
visualize_results_with_colors(training_data, ideal_functions, mapped_results_df, best_ideal_functions)

# Step 5: Unit tests
class TestIdealFunctionSelection(unittest.TestCase):
    def test_ideal_function_selection(self):
        best_functions, _ = select_best_ideal_functions(training_data, ideal_functions)
        self.assertEqual(len(best_functions), 4, "Should select 4 best ideal functions.")

    def test_deviation_calculation(self):
        mapped_results, mapped_data = map_test_data_with_full_log(test_data, ideal_functions, best_ideal_functions, training_data)
        self.assertFalse(mapped_results.empty, "There should be mapped test data points.")

# Run the unit tests
unittest.TextTestRunner().run(unittest.TestLoader().loadTestsFromTestCase(TestIdealFunctionSelection))

# Output paths of the Excel file and visualization HTML file for local access
output_excel_path, r"C:\Users\m.ranka\Desktop\IU\Python\Datasets1\ideal_functions_and_mapping_visualization.html"


..
----------------------------------------------------------------------
Ran 2 tests in 0.189s

OK


('C:\\Users\\m.ranka\\Desktop\\IU\\Python\\Datasets1\\best_ideal_functions_with_full_log.xlsx',
 'C:\\Users\\m.ranka\\Desktop\\IU\\Python\\Datasets1\\ideal_functions_and_mapping_visualization.html')

## Final code where all the requirements as mentioned in assignmnet is met

In [2]:
import pandas as pd
from sklearn.metrics import mean_squared_error
import numpy as np
from bokeh.plotting import figure, output_file, save
from bokeh.models import ColumnDataSource
import os
import webbrowser
import time
from sqlalchemy import create_engine, Column, Integer, Float, String
from sqlalchemy.orm import declarative_base, sessionmaker

# Base Class for Handling Dataset
class DatasetHandler:
    """
    A base class to handle loading and saving datasets.
    
    Attributes:
        file_path (str): Path to the CSV file.
        data (DataFrame): Loaded dataset as a pandas DataFrame.
    """
    def __init__(self, file_path):
        """
        Initializes the DatasetHandler with the given file path.
        
        Args:
            file_path (str): Path to the CSV file.
        """
        self.file_path = file_path
        self.data = None

    def load_data(self):
        """
        Loads data from the CSV file.
        
        Raises:
            FileNotFoundError: If the file is not found.
            pd.errors.EmptyDataError: If the file is empty.
            pd.errors.ParserError: If the file cannot be parsed.
        """
        try:
            self.data = pd.read_csv(self.file_path)
        except FileNotFoundError as e:
            print(f"Error: File not found at {self.file_path}")
            raise e
        except pd.errors.EmptyDataError as e:
            print(f"Error: The file at {self.file_path} is empty.")
            raise e
        except pd.errors.ParserError as e:
            print(f"Error: The file at {self.file_path} could not be parsed.")
            raise e
        except Exception as e:
            print(f"Error: An unexpected error occurred while loading data from {self.file_path}")
            raise e

    def save_data(self, output_path, sheet_name="Sheet1"):
        """
        Saves the DataFrame to an Excel sheet.
        
        Args:
            output_path (str): Path to the output Excel file.
            sheet_name (str): Name of the sheet in the Excel file.
        
        Raises:
            Exception: If an unexpected error occurs while saving.
        """
        if self.data is not None:
            try:
                with pd.ExcelWriter(output_path) as writer:
                    self.data.to_excel(writer, sheet_name=sheet_name, index=False)
            except Exception as e:
                print(f"Error: An unexpected error occurred while saving data to {output_path}")
                raise e

# Derived Classes for Specific Data Types
class TrainingData(DatasetHandler):
    """
    A class to handle training data, inherited from DatasetHandler.
    """
    def __init__(self, file_path):
        super().__init__(file_path)

class IdealFunctions(DatasetHandler):
    """
    A class to handle ideal functions data, inherited from DatasetHandler.
    """
    def __init__(self, file_path):
        super().__init__(file_path)

class TestData(DatasetHandler):
    """
    A class to handle test data, inherited from DatasetHandler.
    """
    def __init__(self, file_path):
        super().__init__(file_path)

class DataMapping:
    """
    A class to handle the mapping of test data to ideal functions.
    
    Attributes:
        training_data (TrainingData): The training data.
        ideal_functions (IdealFunctions): The ideal functions data.
        test_data (TestData): The test data.
        best_functions (dict): Mapping of training functions to ideal functions.
        full_log (DataFrame): Log of the mapping process.
        mapped_data (DataFrame): DataFrame of mapped data points.
    """
    def __init__(self, training_data, ideal_functions, test_data):
        """
        Initializes the DataMapping with training, ideal, and test data.
        
        Args:
            training_data (TrainingData): The training data.
            ideal_functions (IdealFunctions): The ideal functions data.
            test_data (TestData): The test data.
        """
        self.training_data = training_data
        self.ideal_functions = ideal_functions
        self.test_data = test_data
        self.best_functions = None
        self.full_log = None
        self.mapped_data = None

    def select_best_ideal_functions(self):
        """
        Selects the best ideal functions using the Least Squares method.
        
        Raises:
            KeyError: If a required column is missing in the data.
            Exception: If an unexpected error occurs.
        """
        try:
            best_functions = {}
            log_results = []

            for i in range(1, 5):
                y_train = self.training_data.data[f'y{i}'].values
                mse_list = []

                for j in range(1, 51):  # 50 ideal functions
                    y_ideal = self.ideal_functions.data[f'y{j}'].values
                    mse = mean_squared_error(y_train, y_ideal)
                    mse_list.append((mse, f'y{j}'))

                # Find the best ideal function with the lowest MSE
                best_function = min(mse_list, key=lambda x: x[0])
                best_functions[f'y{i}'] = best_function[1]
                log_results.append({
                    'training_function': f'y{i}',
                    'best_ideal_function': best_function[1],
                    'mse': best_function[0]
                })

            # Store the results
            self.best_functions = best_functions
            self.full_log = pd.DataFrame(log_results)
        except KeyError as e:
            print(f"Error: Key error encountered. Please check if all required columns exist in the data.")
            raise e
        except Exception as e:
            print(f"Error: An unexpected error occurred while selecting the best ideal functions.")
            raise e

    def map_test_data(self):
        """
        Maps the test data to the selected ideal functions and logs the process.
        
        Raises:
            ValueError: If best functions are not selected.
            KeyError: If a required column is missing in the data.
            IndexError: If there is an indexing error during mapping.
            Exception: If an unexpected error occurs.
        """
        if self.best_functions is None:
            raise ValueError("Best functions are not selected. Run select_best_ideal_functions() first.")

        try:
            full_log = []
            mapped_data = []

            for index, row in self.test_data.data.iterrows():
                x_test, y_test = row['x'], row['y']
                assigned = False
                log_entry = {
                    'x': x_test,
                    'y_test': y_test,
                    'y_ideal': None,
                    'ideal_function': None,
                    'deviation': None,
                    'threshold': None,
                    'status': 'Not Assigned - Exceeds Threshold'
                }

                for train_key, ideal_key in self.best_functions.items():
                    y_ideal = self.ideal_functions.data[self.ideal_functions.data['x'] == x_test][ideal_key].values
                    if y_ideal.size > 0:
                        y_ideal_value = y_ideal[0]
                        deviation = abs(y_test - y_ideal_value)

                        y_train_value = self.training_data.data[self.training_data.data['x'] == x_test][train_key].values[0]
                        max_deviation_train = abs(y_train_value - y_ideal_value)
                        threshold = np.sqrt(2) * max_deviation_train

                        log_entry.update({
                            'y_ideal': y_ideal_value,
                            'ideal_function': ideal_key,
                            'deviation': deviation,
                            'threshold': threshold
                        })

                        if deviation <= threshold:
                            log_entry['status'] = 'Assigned'
                            mapped_data.append({
                                'x': x_test,
                                'y_test': y_test,
                                'y_ideal': y_ideal_value,
                                'ideal_function': ideal_key,
                                'deviation': deviation,
                                'threshold': threshold,
                                'status': 'Assigned'
                            })
                            assigned = True
                            break

                full_log.append(log_entry)
                if not assigned:
                    mapped_data.append(log_entry)

            self.full_log = pd.DataFrame(full_log)
            self.mapped_data = pd.DataFrame(mapped_data)
        except KeyError as e:
            print(f"Error: Key error encountered during mapping. Please check if all required columns exist in the data.")
            raise e
        except IndexError as e:
            print(f"Error: Index error encountered during mapping. Please check the indexing of your data.")
            raise e
        except Exception as e:
            print(f"Error: An unexpected error occurred while mapping the test data.")
            raise e
    def save_results(self, output_path):
        """
        Saves the mapping results to an Excel file and opens it automatically.
        
        Args:
            output_path (str): Path to the output Excel file.
        
        Raises:
            Exception: If an unexpected error occurs while saving.
        """
        try:
            with pd.ExcelWriter(output_path) as writer:
                if self.full_log is not None:
                    self.full_log.to_excel(writer, sheet_name='Full Log with Deviations', index=False)
                if self.mapped_data is not None:
                    self.mapped_data.to_excel(writer, sheet_name='Mapped Test Data', index=False)

            # Automatically open the Excel file after saving
            os.startfile(output_path)
            time.sleep(2)  # Add delay to give time for the Excel file to open
        except Exception as e:
            print(f"Error: An unexpected error occurred while saving results to {output_path}")
            raise e

# Database Handler Class for SQL Integration
Base = declarative_base()

class DatabaseHandler:
    """
    A class to handle SQL database operations for storing and loading datasets.
    
    Attributes:
        engine (Engine): SQLAlchemy engine for connecting to the SQLite database.
        Session (sessionmaker): Session maker for handling transactions.
    """
    def __init__(self, db_path):
        """
        Initializes the DatabaseHandler with the given database path.
        
        Args:
            db_path (str): Path to the SQLite database.
        """
        self.engine = create_engine(f'sqlite:///{db_path}')
        Base.metadata.create_all(self.engine)
        self.Session = sessionmaker(bind=self.engine)

    def save_dataframe_to_table(self, dataframe, table_name):
        """
        Saves a pandas DataFrame to an SQL table.
        
        Args:
            dataframe (DataFrame): The DataFrame to save.
            table_name (str): The name of the table to save the data in.
        
        Raises:
            Exception: If an unexpected error occurs while saving.
        """
        try:
            dataframe.to_sql(table_name, con=self.engine, if_exists='replace', index=False)
        except Exception as e:
            print(f"Error: An unexpected error occurred while saving DataFrame to the table {table_name}")
            raise e

    def load_table_to_dataframe(self, table_name):
        """
        Loads an SQL table into a pandas DataFrame.
        
        Args:
            table_name (str): The name of the table to load.
        
        Returns:
            DataFrame: Loaded data as a pandas DataFrame.
        
        Raises:
            Exception: If an unexpected error occurs while loading.
        """
        try:
            return pd.read_sql_table(table_name, con=self.engine)
        except Exception as e:
            print(f"Error: An unexpected error occurred while loading table {table_name} into DataFrame.")
            raise e

# Visualization Class
class Visualizer:
    """
    A class to handle visualization of training data, ideal functions, and mapped test data.
    
    Attributes:
        training_data (TrainingData): The training data.
        ideal_functions (IdealFunctions): The ideal functions data.
        mapped_results (DataFrame): The mapped test data.
        best_functions (dict): Mapping of training functions to ideal functions.
    """
    def __init__(self, training_data, ideal_functions, mapped_results, best_functions):
        """
        Initializes the Visualizer with the given data.
        
        Args:
            training_data (TrainingData): The training data.
            ideal_functions (IdealFunctions): The ideal functions data.
            mapped_results (DataFrame): The mapped test data.
            best_functions (dict): The best functions selected for visualization.
        """
        self.training_data = training_data
        self.ideal_functions = ideal_functions
        self.mapped_results = mapped_results
        self.best_functions = best_functions

    def visualize_results(self, output_path):
        """
        Visualizes the training, ideal functions, and mapped test data using Bokeh.
        
        Args:
            output_path (str): Path to the output HTML file for the visualization.
        
        Raises:
            Exception: If an unexpected error occurs during visualization.
        """
        try:
            output_file(output_path)

            p = figure(title="Training Data, Ideal Functions, and Mapped Test Data",
                       x_axis_label="X", y_axis_label="Y", width=800, height=600)

            # Plot training data
            for i in range(1, 5):
                p.scatter(self.training_data.data['x'], self.training_data.data[f'y{i}'], legend_label=f"Training y{i}", size=5, color='blue', alpha=0.6)

            # Plot ideal functions
            for train_key, ideal_key in self.best_functions.items():
                p.line(self.ideal_functions.data['x'], self.ideal_functions.data[ideal_key], legend_label=f"Ideal {ideal_key}", line_width=2, color='green')

            # Plot mapped test data
            source_assigned = ColumnDataSource(self.mapped_results[self.mapped_results['status'] == 'Assigned'])
            source_not_assigned = ColumnDataSource(self.mapped_results[self.mapped_results['status'] != 'Assigned'])

            # Plot points that satisfy the threshold (Assigned) in green
            p.scatter('x', 'y_test', legend_label="Mapped Test Data (Within Threshold)", source=source_assigned, size=8, color='green', alpha=0.6)

            # Plot points that exceed the threshold (Not Assigned) in red
            p.scatter('x', 'y_test', legend_label="Test Data (Exceeds Threshold)", source=source_not_assigned, size=8, color='red', alpha=0.6)

            save(p)
            webbrowser.open(output_path)
        except Exception as e:
            print(f"Error: An unexpected error occurred while visualizing the results.")
            raise e
# Unit Tests
class TestTrainingSystem:
    def __init__(self):
        self.train_file_path = 'train.csv'
        self.ideal_file_path = 'ideal.csv'
        self.test_file_path = 'test.csv'
        self.training_data = TrainingData(self.train_file_path)
        self.ideal_functions = IdealFunctions(self.ideal_file_path)
        self.test_data = TestData(self.test_file_path)

    def test_load_data(self):
        """Test loading of data."""
        self.training_data.load_data()
        self.ideal_functions.load_data()
        self.test_data.load_data()
        assert self.training_data.data is not None, "Training data should be loaded."
        assert self.ideal_functions.data is not None, "Ideal functions data should be loaded."
        assert self.test_data.data is not None, "Test data should be loaded."
        print("test_load_data passed")

    def test_select_best_ideal_functions(self):
        """Test selection of best ideal functions."""
        self.training_data.load_data()
        self.ideal_functions.load_data()
        data_mapping = DataMapping(self.training_data, self.ideal_functions, self.test_data)
        data_mapping.select_best_ideal_functions()
        assert data_mapping.best_functions is not None, "Best functions should be selected."
        assert len(data_mapping.best_functions) == 4, "There should be 4 best functions selected."
        print("test_select_best_ideal_functions passed")

    def test_map_test_data(self):
        """Test mapping of test data to ideal functions."""
        self.training_data.load_data()
        self.ideal_functions.load_data()
        self.test_data.load_data()
        data_mapping = DataMapping(self.training_data, self.ideal_functions, self.test_data)
        data_mapping.select_best_ideal_functions()
        data_mapping.map_test_data()
        assert data_mapping.mapped_data is not None, "Mapped data should not be None."
        assert len(data_mapping.mapped_data) > 0, "There should be mapped data points."
        print("test_map_test_data passed")

# Main function
def main():
    # File paths in the current working directory
    train_file_path = 'train.csv'
    ideal_file_path = 'ideal.csv'
    test_file_path = 'test.csv'
    output_excel_path = 'best_ideal_functions_with_full_log.xlsx'
    output_html_path = 'visualization.html'
    db_path = 'data.db'
    output_db_excel_path = 'db_data_output.xlsx'

    # Load datasets
    training_data = TrainingData(train_file_path)
    training_data.load_data()

    ideal_functions = IdealFunctions(ideal_file_path)
    ideal_functions.load_data()

    test_data = TestData(test_file_path)
    test_data.load_data()

    # Data mapping
    data_mapping = DataMapping(training_data, ideal_functions, test_data)
    data_mapping.select_best_ideal_functions()
    data_mapping.map_test_data()
    data_mapping.save_results(output_excel_path)

    # Save data to SQL Database
    db_handler = DatabaseHandler(db_path)
    db_handler.save_dataframe_to_table(training_data.data, "TrainingData")
    db_handler.save_dataframe_to_table(ideal_functions.data, "IdealFunctions")
    db_handler.save_dataframe_to_table(test_data.data, "TestData")
    db_handler.save_dataframe_to_table(data_mapping.mapped_data, "MappedTestData")
    db_handler.save_dataframe_to_table(data_mapping.full_log, "FullLog")

    # Load from database to a new Excel file
    loaded_training_data = db_handler.load_table_to_dataframe("TrainingData")
    loaded_ideal_functions = db_handler.load_table_to_dataframe("IdealFunctions")
    loaded_test_data = db_handler.load_table_to_dataframe("TestData")
    loaded_mapped_data = db_handler.load_table_to_dataframe("MappedTestData")
    loaded_full_log = db_handler.load_table_to_dataframe("FullLog")

    # Save loaded data into a new Excel file with multiple sheets
    with pd.ExcelWriter(output_db_excel_path) as writer:
        loaded_training_data.to_excel(writer, sheet_name='Training Data', index=False)
        loaded_ideal_functions.to_excel(writer, sheet_name='Ideal Functions', index=False)
        loaded_test_data.to_excel(writer, sheet_name='Test Data', index=False)
        loaded_mapped_data.to_excel(writer, sheet_name='Mapped Test Data', index=False)
        loaded_full_log.to_excel(writer, sheet_name='Full Log', index=False)

    # Automatically open the Excel file after saving
    os.startfile(output_db_excel_path)
    time.sleep(2)  # Add delay to give time for the Excel file to open

    # Visualization
    visualizer = Visualizer(training_data, ideal_functions, data_mapping.mapped_data, data_mapping.best_functions)
    visualizer.visualize_results(output_html_path)

def run_tests():
    tests = TestTrainingSystem()
    tests.test_load_data()
    tests.test_select_best_ideal_functions()
    tests.test_map_test_data()

if __name__ == "__main__":
    # Choose whether to run tests or main function
    run_tests()
    main()


test_load_data passed
test_select_best_ideal_functions passed
test_map_test_data passed
